In [1]:
import os
import sys
import pickle
import numpy as np
import tensorflow as tf

from sklearn import svm
from sklearn import tree
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA 
from sklearn.utils import shuffle
from sklearn.externals import joblib

sys.path.insert(0, "/home/anonymous/Desktop/eeg_sampling/modeling")
from dataio import readdata, readlabels, writedata

In [11]:
#Read training
train_dataset = readdata("./../curated/train_dataset")
train_labels = readlabels("./../curated/train_labels")
#Read validation
#valid_dataset = readdata("./curated/valid_dataset")
#valid_labels = readlabels("./curated/valid_labels")
#Read test
test_dataset = readdata("./../curated/test_dataset")
test_labels = readlabels("./../curated/test_labels")

print('Training:', train_dataset.shape, train_labels.shape)
#print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (1218, 360, 8) (1218,)
Testing: (73, 360, 8) (73,)


In [12]:
n_input = 8*360 # EEG data input (8 channels * 360 sample points)
n_classes = 5 # EEG total classes ("nothing", "up", "down", "left", "right")

In [13]:
train_dataset.shape = (train_dataset.shape[0], n_input)
test_dataset.shape = (test_dataset.shape[0], n_input)

print(train_dataset.shape)
print(test_dataset.shape)

(1218, 2880)
(73, 2880)


In [25]:
#KNN
knn = KNeighborsClassifier(
    algorithm="auto", 
    weights="uniform", 
    n_neighbors=50)
knn.fit(train_dataset, train_labels)
knn_pred = knn.predict(test_dataset)
knn_acc = accuracy_score(test_labels, knn_pred)
print("Knn Acc: ", knn_acc)
print(knn_pred[0:37])
print(test_labels[0:37])

Knn Acc:  0.849315068493
[3 4 0 3 1 0 2 3 2 0 3 2 3 2 0 2 2 1 3 2 2 2 3 2 1 2 0 1 2 0 3 3 0 4 1 3 4]
[3 4 0 3 1 0 2 3 2 0 3 2 3 2 0 2 4 1 3 2 4 2 3 2 1 2 0 2 4 0 4 3 0 3 2 3 4]


In [15]:
#LDA
lda = LinearDiscriminantAnalysis(
    solver="svd",
    store_covariance=False,
    tol = 0.0001)
lda.fit(train_dataset, train_labels)
lda_pred = lda.predict(test_dataset)
lda_acc = accuracy_score(test_labels, lda_pred)
print("Lda Acc: ", lda_acc)


Lda Acc:  0.27397260274


/usr/lib64/python3.4/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [26]:
#Random Forests
forest = ExtraTreesClassifier(n_estimators = 1000)
forest.fit(train_dataset, train_labels)
forest_pred = forest.predict(test_dataset)
forest_acc = accuracy_score(test_labels, forest_pred)
print("For Acc: ", forest_acc)
print(forest_pred[0:37])
print(test_labels[0:37])

For Acc:  0.821917808219
[3 4 0 3 1 0 2 0 2 0 3 2 3 4 0 3 3 1 3 2 3 2 3 2 1 2 0 1 2 3 3 3 0 3 1 3 4]
[3 4 0 3 1 0 2 3 2 0 3 2 3 2 0 2 4 1 3 2 4 2 3 2 1 2 0 2 4 0 4 3 0 3 2 3 4]


In [27]:
#SVM
svm_mod = svm.LinearSVC(
    C=1.0,
    penalty="l2",
    loss="squared_hinge",
    tol=0.0001)
svm_mod.fit(train_dataset, train_labels)
svm_pred = svm_mod.predict(test_dataset)
svm_acc = accuracy_score(test_labels, svm_pred)
print("Svm Acc: ", svm_acc)
print(forest_pred[0:37])
print(test_labels[0:37])

Svm Acc:  0.465753424658
[3 4 0 3 1 0 2 0 2 0 3 2 3 4 0 3 3 1 3 2 3 2 3 2 1 2 0 1 2 3 3 3 0 3 1 3 4]
[3 4 0 3 1 0 2 3 2 0 3 2 3 2 0 2 4 1 3 2 4 2 3 2 1 2 0 2 4 0 4 3 0 3 2 3 4]


In [9]:
#Ada
ada = AdaBoostClassifier(n_estimators = 50)
ada.fit(train_dataset, train_labels)
ada_pred = ada.predict(test_dataset)
ada_acc = accuracy_score(test_labels, ada_pred)
print("Ada Acc: ", ada_acc)

Ada Acc:  0.0333333333333


In [10]:
#Logistic
lgr = linear_model.LogisticRegression()
lgr.fit(train_dataset, train_labels)
lgr_pred = lgr.predict(test_dataset)
lgr_acc = accuracy_score(test_labels, lgr_pred)
print("Lgr Acc: ", lgr_acc)

Lgr Acc:  0.1


In [ ]:
#Save Model
joblib.dump(forest, './predicting/model/eeg.model')

In [ ]:
#Read Model
forest = joblib.load('./predicting/model/eeg.model') 

In [ ]:
forest_pred = forest.predict(test_dataset)
forest_acc = accuracy_score(test_labels, forest_pred)
print(forest_acc)

In [ ]:
#Voting
vot = VotingClassifier(estimators=[
        ("KNN", knn),
        ("LDA", lda),
        ("FOR", forest),
        ("SVM", svm_mod),
        ("ADA", ada),
        ("LGR", lgr)
    ], voting = "hard", weights=[6,0,5,1,1,1])
vot.fit(train_dataset, train_labels)
vot_pred = vot.predict(test_dataset)
print(accuracy_score(test_labels, vot_pred))
print(vot_pred)
print(test_labels)

In [ ]:
# save the classifier
with open('./predicting/eeg.model', 'wb') as writestream:
    pickle.dump(forest, writestream)

In [ ]:
import pickle
from dataio import readdata, readlabels, writedata
# load it again
with open('./predicting/eeg.model', 'rb') as readstream:
    clf = pickle.load(readstream)

In [ ]:
clf_pred = clf.predict(train_dataset)
print(clf_pred)
print(train_labels)
#print(accuracy_score(train_labels, clf_pred))

In [ ]:
#PCA 
pca = PCA(n_components=20)
train_dataset = pca.fit_transform(train_dataset)
test_dataset = pca.fit_transform(test_dataset)
print(train_dataset.shape)
print(test_dataset.shape)

In [ ]:
#SGD
clf = SGDClassifier(loss="hinge", penalty="l2")
clf.fit(train_dataset, train_labels)
clf_pred = clf.predict(test_dataset)
accuracy_score(test_labels, clf_pred)

In [ ]:
#Logistic
clf = linear_model.Perceptron(n_iter=100)
clf.fit(train_dataset, train_labels)
clf_pred = clf.predict(test_dataset)
clf_acc = accuracy_score(test_labels, clf_pred)
print(clf_acc)

In [ ]:
#Bags
knn = BaggingClassifier(KNeighborsClassifier(
    algorithm="auto", 
    weights="uniform", 
    n_neighbors=15), max_samples=0.5, max_features=0.5)
knn.fit(train_dataset, train_labels)
knn_pred = knn.predict(test_dataset)
knn_acc = accuracy_score(test_labels, knn_pred)
print(knn_acc)